In [32]:
%autosave 0
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Téléchargez la liste de stop words en français
nltk.download('stopwords')
nltk.download('punkt')

# Importez la liste de stop words en français
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df_test = pd.read_excel("Data/df_complet_clean.xlsx")
df_test.head(10)

,Unnamed: 0.1,Unnamed: 0,Client,Nombre_avis_publie,Note,Titres,Commentaire,Date_experience,Date_publication,Reponse,Date_reponse,Pays,Verifications,Entreprise
0,0,0,marc lesterpt,1,5,Je commande mes pneus de motos et de voitures,Je commande mes pneus de motos et de voitures ...,2023-07-23,2023-08-05,NaN,NaT,GR,1,Allopneus
1,1,1,Séverine M,6,5,"Ne cherchez plus vos pneus, Allopneus le peut","BonjourLe site est genial, tres bien explique ...",2023-07-13,2023-08-04,NaN,NaT,FR,1,Allopneus
2,2,2,PINTO NUNO,3,5,Tout etait parfait,"Tout etait parfait, timing parfait dans les da...",2023-07-16,2023-08-04,NaN,NaT,FR,1,Allopneus
3,3,3,client,4,5,Pneus d'un tres bon rapport qualite_,Pneus d'un tres bon rapport qualite prix. Les ...,2023-07-24,2023-08-05,NaN,NaT,FR,1,Allopneus
4,4,4,Vignaux,1,4,"Bon suivi de la commande,les questions_","Bon suivi de la commande,les questions sont im...",2023-07-23,2023-08-04,NaN,NaT,FR,1,Allopneus
5,5,5,cliente juliema,1,5,"Excellent service, bien plus avantageux compar...",Ce service sur place est d un grand secours. R...,2023-07-24,2023-08-04,NaN,NaT,FR,1,Allopneus
6,6,6,cliente,1,5,commande jusqu'au montage,La commande sur internet c'est tres bien passe...,2023-07-11,2023-08-01,NaN,NaT,FR,1,Allopneus
7,7,7,Manu,4,5,Excellent,"RAS, le rapport qualite prix est tres correct....",2023-07-26,2023-08-05,NaN,NaT,FR,1,Allopneus
8,8,8,Xavier,3,5,Fidele depuis de nombreuses annees j ai_,Fidele depuis de nombreuses annees j ai toujou...,2023-07-20,2023-08-04,NaN,NaT,FR,1,Allopneus
9,9,9,Sonia Nutrition,1,5,Meilleur rapport qualite prix,M'a ete recommande. Meilleur rapport qualite p...,2023-07-20,2023-08-04,NaN,NaT,FR,1,Allopneus


In [34]:
# (a) Supprimer les caractères spéciaux, chiffres, valeurs manquantes et mettre le texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Titres'] = df_test['Titres'].fillna('') 
df_test = df_test.drop(columns=['Unnamed: 0.1','Unnamed: 0','Reponse','Date_reponse','Pays'])
df_test.head(10)

C:\Users\joan\AppData\Local\Temp\ipykernel_7764\3216779368.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
C:\Users\joan\AppData\Local\Temp\ipykernel_7764\3216779368.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()


KeyError: "['Unnamed: 0.1', 'Unnamed: 0', 'Reponse', 'Date_reponse', 'Pays'] not found in axis"

In [9]:
from gensim.models import FastText

# Replace 'path/to/pretrained/embeddings' with the actual path to your downloaded embeddings file
model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")

C:\Users\joan\AppData\Local\Temp\ipykernel_7764\2105882537.py:4: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")


In [43]:
# Define preprocessing function
stop_words = set(stopwords.words('french'))

def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

# Apply FastText embeddings to each element in the Series
def get_text_embedding(text):
    preprocessed_text = preprocess_text(text)
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    
    if word_embeddings:
        text_embedding = np.mean(word_embeddings, axis=0)  # Use mean for aggregation
    else:
        text_embedding = np.zeros(model.vector_size)  # Use zero vector if no embeddings found
    
    return text_embedding


In [44]:
# Create a new column 'embedding' with the text embeddings
df_test['embedding_Commentaire'] = df_test['Commentaire'].apply(get_text_embedding)
df_test['embedding_Titres'] = df_test['Titres'].apply(get_text_embedding)

In [67]:
print(X_train.tolist()[0:5])
#print([x for x in X_train.apply(lambda x: len(x)) if x != 300])

[array([ 6.82445336e-03,  9.94043238e-03, -2.31176130e-02, -1.64629724e-02,
       -1.25230044e-01, -2.55642738e-02, -1.09481476e-02, -1.19698308e-02,
       -3.28704119e-02, -5.10599390e-02,  2.46275030e-02, -2.28163656e-02,
        1.15908086e-02, -1.78757478e-02,  2.87410207e-02, -3.06206290e-02,
        1.52352899e-02,  1.13705657e-02,  1.57539230e-02,  3.44328992e-02,
        2.43450403e-02, -2.28232127e-02,  4.69645625e-03, -5.95921697e-03,
        4.27637026e-02,  2.49656513e-02, -4.90378663e-02, -4.79459614e-02,
       -1.29721034e-02,  8.13726038e-02,  5.38131734e-03,  2.07741465e-02,
        4.55551967e-03,  6.38498291e-02,  1.06230527e-02, -1.84797272e-02,
        7.01133767e-03,  1.52330101e-02, -2.13812813e-02, -4.46230248e-02,
       -3.47778052e-02, -7.36945542e-04,  1.97512563e-02, -1.24026882e-02,
       -9.94983166e-02, -5.25464583e-03,  8.53184424e-03, -3.03277466e-02,
        1.40944927e-03,  2.04791641e-03,  2.83495970e-02,  2.36934684e-02,
        8.62381794e-03, 

In [68]:
# (b) Division du DataFrame en ensembles d'entraînement et de test.
X_train, X_test, y_train, y_test = train_test_split(df_test['embedding_Commentaire'], df_test['Note'], test_size=0.2, random_state=42)

In [69]:
# For future use it would be interesting to use numpy to transpose rows and columns and then create a dataframe
X_train = X_train.tolist()
X_test = X_test.tolist()

In [70]:
# (d) Création du classificateur RandomForest.
clf = make_pipeline(StandardScaler(with_mean=False), RandomForestClassifier(random_state=42))

In [71]:
# (e) Entraînement du classificateur sur l'ensemble d'entraînement.
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [72]:
# (f) Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

           1       0.57      0.62      0.60      5944
           2       0.14      0.00      0.00      1936
           3       0.32      0.02      0.03      2765
           4       0.42      0.08      0.13      8656
           5       0.76      0.98      0.85     37896

    accuracy                           0.72     57197
   macro avg       0.44      0.34      0.32     57197
weighted avg       0.64      0.72      0.65     57197



In [73]:
# (g) Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Classe prédite,1,2,3,4,5
Classe réelle,,,,,
1,3708,9,28,119,2080
2,900,3,33,96,904
3,910,1,44,241,1569
4,567,2,25,683,7379
5,420,6,7,479,36984


# additionnal libraries

In [ ]:
from keras import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout

# Word Embedding 1: bag of word

In [ ]:
embedding_dim = 300
# Model
model = Sequential()
# 1. Embedding
model.add(Embedding(10000, embedding_dim))
# 2. Bag of Words
model.add(GlobalAveragePooling1D())
# 3. Classification
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit
history = model.fit(X_train, y_train,
    batch_size = 32,
    epochs = 5,
    validation_data = [X_test, y_test])

# Word Embedding 2: Vectorizer

In [ ]:
embedding_dim = 300
# Model
model = Sequential()
# 1. Embedding
model.add(Embedding(10000, embedding_dim))
# 2. Bag of Words
model.add(GlobalAveragePooling1D())
# 3. Classification
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit
history = model.fit(X_train, y_train,
    batch_size = 32,
    epochs = 5,
    validation_data = [X_test, y_test])